In [6]:
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 0. Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# 0.1. Enable GPU memory growth if available
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 2. Data Generators
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    'hollywood_train',  # Update path to match your directory structure
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)
val_gen = val_datagen.flow_from_directory(
    'hollywood_validation',  # Update path to match your directory structure
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Print class information
print(f"Number of training classes: {train_gen.num_classes}")
print(f"Training class indices: {train_gen.class_indices}")
print(f"Number of validation classes: {val_gen.num_classes}")
print(f"Validation class indices: {val_gen.class_indices}")

# 3. Model Building (Transfer Learning)
num_classes = train_gen.num_classes
base_model = VGG19(weights='imagenet', include_top=False, input_shape=img_size + (3,))

for layer in base_model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 4. Training with Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy')
]

history = model.fit(
    train_gen,
    epochs=25,
    validation_data=val_gen,
    callbacks=callbacks
)

# 5. Evaluation
loss, acc = model.evaluate(val_gen)
print(f"Validation Loss: {loss:.4f}, Validation Accuracy: {acc:.4f}")

# 5.1. Detailed Classification Report
y_true = val_gen.classes
val_gen.reset()
Y_pred = model.predict(val_gen)
y_pred = np.argmax(Y_pred, axis=1)
print(classification_report(y_true, y_pred, target_names=list(val_gen.class_indices.keys())))

# 6. Save the final model
model.save('final_model.h5')

Found 1440 images belonging to 17 classes.
Found 360 images belonging to 17 classes.
Number of training classes: 17
Training class indices: {'Angelina Jolie': 0, 'Brad Pitt': 1, 'Denzel Washington': 2, 'Hugh Jackman': 3, 'Jennifer Lawrence': 4, 'Johnny Depp': 5, 'Kate Winslet': 6, 'Leonardo DiCaprio': 7, 'Megan Fox': 8, 'Natalie Portman': 9, 'Nicole Kidman': 10, 'Robert Downey Jr': 11, 'Sandra Bullock': 12, 'Scarlett Johansson': 13, 'Tom Cruise': 14, 'Tom Hanks': 15, 'Will Smith': 16}
Number of validation classes: 17
Validation class indices: {'Angelina Jolie': 0, 'Brad Pitt': 1, 'Denzel Washington': 2, 'Hugh Jackman': 3, 'Jennifer Lawrence': 4, 'Johnny Depp': 5, 'Kate Winslet': 6, 'Leonardo DiCaprio': 7, 'Megan Fox': 8, 'Natalie Portman': 9, 'Nicole Kidman': 10, 'Robert Downey Jr': 11, 'Sandra Bullock': 12, 'Scarlett Johansson': 13, 'Tom Cruise': 14, 'Tom Hanks': 15, 'Will Smith': 16}


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 20,160,081 (76.90 MB)

 Trainable params: 135,697 (530.07 KB)

...

45/45 ━━━━━━━━━━━━━━━━━━━━ 184s 4s/step - accuracy: 0.3242 - loss: 2.0933 - val_accuracy: 0.3528 - val_loss: 2.0285
Epoch 21/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3190 - loss: 2.0913

45/45 ━━━━━━━━━━━━━━━━━━━━ 182s 4s/step - accuracy: 0.3188 - loss: 2.0915 - val_accuracy: 0.3722 - val_loss: 2.0187
Epoch 22/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 185s 4s/step - accuracy: 0.3180 - loss: 2.0927 - val_accuracy: 0.3694 - val_loss: 2.0106
Epoch 23/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3312 - loss: 2.0646

45/45 ━━━━━━━━━━━━━━━━━━━━ 184s 4s/step - accuracy: 0.3314 - loss: 2.0639 - val_accuracy: 0.3778 - val_loss: 1.9875
Epoch 24/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 186s 4s/step - accuracy: 0.3617 - loss: 1.9548 - val_accuracy: 0.3778 - val_loss: 1.9745
Epoch 25/25
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3420 - loss: 1.9896

45/45 ━━━━━━━━━━━━━━━━━━━━ 177s 4s/step - accuracy: 0.3420 - loss: 1.9895 - val_accuracy: 0.3972 - val_loss: 1.9494
12/12 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step - accuracy: 0.3800 - loss: 1.9488
Validation Loss: 1.9494, Validation Accuracy: 0.3972
12/12 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step


                    precision    recall  f1-score   support

    Angelina Jolie       0.00      0.00      0.00        20
         Brad Pitt       0.00      0.00      0.00        20
 Denzel Washington       0.05      0.05      0.05        20
      Hugh Jackman       0.00      0.00      0.00        20
 Jennifer Lawrence       0.06      0.05      0.05        20
       Johnny Depp       0.09      0.10      0.10        20
      Kate Winslet       0.06      0.05      0.06        20
 Leonardo DiCaprio       0.00      0.00      0.00        20
         Megan Fox       0.04      0.05      0.04        20
   Natalie Portman       0.14      0.20      0.17        20
     Nicole Kidman       0.10      0.10      0.10        20
  Robert Downey Jr       0.05      0.10      0.06        20
    Sandra Bullock       0.00      0.00      0.00        20
Scarlett Johansson       0.20      0.25      0.22        40
        Tom Cruise       0.00      0.00      0.00        20
         Tom Hanks       0.00      0.00